In [ ]:
pip install apyori

  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5955 sha256=312fd5951766745333299c663f10d81c9cb9816bada25174ce4fe194c9d1c554
  Stored in directory: /root/.cache/pip/wheels/c4/1a/79/20f55c470a50bb3702a8cb7c94d8ada15573538c7f4baebe2d
Successfully built apyori


In [ ]:
import numpy as np
import pandas as pd
from apyori import apriori

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Groceries_dataset.csv to Groceries_dataset.csv


In [ ]:
import io
data = pd.read_csv(io.BytesIO(uploaded['Groceries_dataset.csv']))

In [ ]:
data

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk
...,...,...,...
38760,4471,08-10-2014,sliced cheese
38761,2022,23-02-2014,candy
38762,1097,16-04-2014,cake bar
38763,1510,03-12-2014,fruit/vegetable juice


In [ ]:
#Sampling #َApriori
sample = data.sample(frac =.5)
print(sample.head())
# sample size
print(f'The sample have {sample.shape[0]} rows and {sample.shape[1]} columns')

transactions = sample.groupby(['Member_number', 'Date'])
transactions.count().reset_index

transactions_list = []
transactions_list = [i[1]['itemDescription'].tolist() for i in list(transactions)]

association_rules = apriori(transactions = transactions_list , min_support=0.001, min_confidence=0.05, min_lift=1.2)
results = list(association_rules)

pd.options.display.float_format = '{:,.2f}'.format
rules_df = pd.DataFrame(columns = ['Left Hand Side', 'Right Hand Side', 'Support(%)', 'Confidence(%)', 'Lift'])
for i in results:
    if len(i[0]) > 1:
        for j in range(0, len(i[2])):
            LHS = list(i[2][j][0])[0]
            RHS = list(i[2][j][1])[0]
            SUPPORT = i[1]*100
            CONFIDENCE = i[2][j][2]*100
            LIFT = i[2][j][3]
            new_row = {'Left Hand Side': LHS, 'Right Hand Side': RHS, 'Support(%)': SUPPORT, 'Confidence(%)': CONFIDENCE, 'Lift': LIFT}
            rules_df = rules_df.append(new_row, ignore_index = True)
rules_df['Rule'] = rules_df['Left Hand Side'] + ' -> ' + rules_df['Right Hand Side']
apriori_rules = rules_df
apriori_rules.head()


       Member_number        Date   itemDescription
6514            2897  27-01-2015          zwieback
5755            3063  28-12-2015  processed cheese
22457           3280  14-04-2015    tropical fruit
22911           2772  04-12-2015           napkins
17212           2125  25-06-2015      frozen meals
The sample have 19382 rows and 3 columns


<ipython-input-7-f35dd7c3ee1e>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append(new_row, ignore_index = True)
<ipython-input-7-f35dd7c3ee1e>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append(new_row, ignore_index = True)
<ipython-input-7-f35dd7c3ee1e>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append(new_row, ignore_index = True)
<ipython-input-7-f35dd7c3ee1e>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append(new_row, ignore_index = True)


,Left Hand Side,Right Hand Side,Support(%),Confidence(%),Lift,Rule
0,chicken,soda,0.14,8.25,1.27,chicken -> soda
1,fruit/vegetable juice,rolls/buns,0.17,8.55,1.22,fruit/vegetable juice -> rolls/buns
2,napkins,soda,0.11,8.12,1.25,napkins -> soda
3,salty snack,rolls/buns,0.11,9.09,1.30,salty snack -> rolls/buns


In [ ]:
pip install mlxtend

In [ ]:
#Sampling #FP
sample = data.sample(frac =.5)
print(sample.head())
# sample size
print(f'The sample have {sample.shape[0]} rows and {sample.shape[1]} columns')

transactions = sample.groupby(['Member_number', 'Date'])
transactions.count().reset_index

from mlxtend.preprocessing import TransactionEncoder
fp_data = transactions['itemDescription'].apply(list).to_list()
te = TransactionEncoder()
te.fit(fp_data)
enc_transactions = te.transform(fp_data)

fp_transactions = pd.DataFrame(enc_transactions, columns = te.columns_)
fp_transactions.head()

from mlxtend.frequent_patterns.fpgrowth import fpgrowth
frequent_itemsets = fpgrowth(fp_transactions, min_support=0.008, use_colnames = True)

from mlxtend.frequent_patterns import association_rules

fp_rules = association_rules(frequent_itemsets, metric = "confidence", min_threshold=0.05)
fp_rules["antecedents"] = fp_rules["antecedents"].apply(lambda x: list(x)[0])
fp_rules["consequents"] = fp_rules["consequents"].apply(lambda x: list(x)[0])
fp_rules['Rule'] = fp_rules['antecedents'] + ' -> ' + fp_rules['consequents']
fp_rules


       Member_number        Date   itemDescription
9186            1621  18-06-2014  other vegetables
10932           1395  24-11-2014           sausage
22302           2699  28-11-2015            pastry
6538            4885  16-10-2015        whole milk
32222           2599  30-07-2015            yogurt
The sample have 19382 rows and 3 columns


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,Rule


In [ ]:
#Sampling #FP
sample = data.sample(frac =.8)
print(sample.head())
# sample size
print(f'The sample have {sample.shape[0]} rows and {sample.shape[1]} columns')

transactions = sample.groupby(['Member_number', 'Date'])
transactions.count().reset_index

fp_data = transactions['itemDescription'].apply(list).to_list()
te = TransactionEncoder()
te.fit(fp_data)
enc_transactions = te.transform(fp_data)

fp_transactions = pd.DataFrame(enc_transactions, columns = te.columns_)
fp_transactions.head()

frequent_itemsets = fpgrowth(fp_transactions, min_support=0.008, use_colnames = True)

from mlxtend.frequent_patterns import association_rules

fp_rules = association_rules(frequent_itemsets, metric = "confidence", min_threshold=0.05)
fp_rules["antecedents"] = fp_rules["antecedents"].apply(lambda x: list(x)[0])
fp_rules["consequents"] = fp_rules["consequents"].apply(lambda x: list(x)[0])
fp_rules['Rule'] = fp_rules['antecedents'] + ' -> ' + fp_rules['consequents']
fp_rules


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


       Member_number        Date  itemDescription
15102           4059  13-06-2014         UHT-milk
15084           3631  12-09-2014  root vegetables
2970            2794  29-09-2015          sausage
32931           1685  18-10-2014       whole milk
17219           3549  01-08-2015  misc. beverages
The sample have 31012 rows and 3 columns


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,Rule
0,rolls/buns,whole milk,0.09,0.13,0.01,0.09,0.73,-0.00,0.96,-0.29,rolls/buns -> whole milk
1,whole milk,rolls/buns,0.13,0.09,0.01,0.07,0.73,-0.00,0.97,-0.30,whole milk -> rolls/buns
2,other vegetables,whole milk,0.10,0.13,0.01,0.09,0.73,-0.00,0.96,-0.29,other vegetables -> whole milk
3,whole milk,other vegetables,0.13,0.10,0.01,0.07,0.73,-0.00,0.97,-0.30,whole milk -> other vegetables
